# 生存label

In [31]:
from onekey_algo import get_param_in_cwd
from onekey_algo.custom.components.metrics import get_time_dependent_gt
import pandas as pd
import os

root = get_param_in_cwd('radio_dir')
data = pd.read_csv(os.path.join(root, 'data.csv'))[['ID', 'OS', 'OSTIME']]
label_data = get_time_dependent_gt(data, 36, duration_col='OSTIME', event_col='OS')
label_data['group'] = label_data['ID'].map(lambda x: 'train' if 'primary' in x else 'test')
label_data.to_csv(os.path.join(root, 'label.csv'), index=False)
print(label_data[['group', 'label']].value_counts(), label_data.shape)

group  label
train  0        331
test   0         78
train  1         59
test   1         21
dtype: int64 (489, 3)


# 形成2.5D数据

In [32]:
samples = []
for idx, row in label_data.iterrows():
    label = row['label']
    fname = row['ID']
    for islice in ['', '_+01', '_+02', '_+04', '_-01', '_-02', '_-04']:
        samples.append([fname, fname.replace('.nii.gz', f'.nii{islice}.png'), label])
samples = pd.DataFrame(samples, columns=['ID', 'fpath', 'label'])
samples

,ID,fpath,label
0,primary-1.nii.gz,primary-1.nii.png,1
1,primary-1.nii.gz,primary-1.nii_+01.png,1
2,primary-1.nii.gz,primary-1.nii_+02.png,1
3,primary-1.nii.gz,primary-1.nii_+04.png,1
4,primary-1.nii.gz,primary-1.nii_-01.png,1
...,...,...,...
3418,secondary2-72.nii.gz,secondary2-72.nii_+02.png,0
3419,secondary2-72.nii.gz,secondary2-72.nii_+04.png,0
3420,secondary2-72.nii.gz,secondary2-72.nii_-01.png,0
3421,secondary2-72.nii.gz,secondary2-72.nii_-02.png,0


In [33]:
import os
import re
import shutil
import pandas as pd
from onekey_algo.custom.components.comp2 import split_dataset4sol
from onekey_algo import get_param_in_cwd
from onekey_algo.custom.utils import print_join_info
import numpy as np

survival_data = pd.read_csv(get_param_in_cwd('survival_file'))
train_data = survival_data[survival_data['group'] == 'train']
test_data = survival_data[survival_data['group'] != 'train']

rt = split_dataset4sol(train_data, train_data['OS'], cv=False, n_trails=10, test_size=0.33, save_dir='.', shuffle=True)
for idx, (train, val) in enumerate(rt):
    val['group'] = 'val'
    c_data = pd.concat([train, val, test_data], axis=0)
    c_data.to_csv(f'split_info/label-RND-{idx}.csv', index=False)
    
    # 2.5D 数据
    tr_data = pd.merge(samples, c_data[c_data['group'] == 'train']['ID'], on='ID', how='inner')
    tr_data[['fpath', 'label']].to_csv(f'split_info/train-RND-{idx}.txt', sep='\t', index=False, header=False)
    te_data =  pd.merge(samples, c_data[c_data['group'] != 'train']['ID'], on='ID', how='inner')
    te_data[['fpath', 'label']].to_csv(f'split_info/val-RND-{idx}.txt', sep='\t', index=False, header=False)
    print(f"训练集：{len(tr_data)}|{len(np.unique(tr_data['ID']))}, 非训练集：{len(te_data)}|{len(np.unique(te_data['ID']))}")

训练集：1841|263, 非训练集：1582|226
训练集：1820|260, 非训练集：1603|229
训练集：1806|258, 非训练集：1617|231
训练集：1827|261, 非训练集：1596|228
训练集：1820|260, 非训练集：1603|229
训练集：1848|264, 非训练集：1575|225
训练集：1841|263, 非训练集：1582|226
训练集：1848|264, 非训练集：1575|225
训练集：1820|260, 非训练集：1603|229
训练集：1813|259, 非训练集：1610|230
